# Retrival-Augmented Generation (RAG) Example

This notebook demonstrates how to use Retrieval-Augmented Generation (RAG) to improve responses using the OpenAI API.

First, copy this lab to your notebooks folder.

## Step 1: Install and Import Libraries

In [1]:
from dotenv import dotenv_values
import openai
from openai import OpenAI
print(f'Using openai {openai.__version__}')

Using openai 1.37.1



## Step 2: Set Up API Key


In [2]:
config = dotenv_values()
openai_api_key = config['OPENAI_API_KEY']
client = OpenAI(
    # This is the default and can be omitted
    api_key=openai_api_key,
)

# Define a function to get a response from OpenAI API
def get_openai_response(client, input_text, model):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": input_text,
            }
        ],
        model=model,
    )
    return chat_completion.choices[0].message.content.strip()


## Step 3: Create a List of Facts

In [3]:
# Create a list of facts
facts = [
    "The Eiffel Tower is in Paris.",
    "The Great Wall of China can be seen from space.",
    "The largest mammal is the blue whale.",
    "Venus is the hottest planet in our solar system.",
    "Honey never spoils."
]
facts_words = [fact.lower().strip('.').split(' ') for fact in facts]
facts_words

[['the', 'eiffel', 'tower', 'is', 'in', 'paris'],
 ['the', 'great', 'wall', 'of', 'china', 'can', 'be', 'seen', 'from', 'space'],
 ['the', 'largest', 'mammal', 'is', 'the', 'blue', 'whale'],
 ['venus', 'is', 'the', 'hottest', 'planet', 'in', 'our', 'solar', 'system'],
 ['honey', 'never', 'spoils']]


## Step 4: Define a Simple Retrieval Function

In [4]:
# Function to retrieve relevant facts
# This very simple toy example returns a list of facts that have any relavant word from query.
# In the real world you would use search with links to a document store.
def retrieve_facts(query, facts):
    facts_words = [fact.lower().strip('.').split(' ') for fact in facts]
    exclude = ['a','the','of','is']
    query_words = [word for word in query.lower().strip('.').split(' ') if word not in exclude]
    relevant_facts = {} # dict keys only allow unique key values
    for query_word in query_words:
        for fact, fact_words in zip(facts, facts_words):
            for query_word in query_words:
                if query_word in fact_words:
                    relevant_facts[fact] = '' 
                    break
    return list(relevant_facts.keys())

# Test the function
retrieve_facts("the hottest tower", facts)


['The Eiffel Tower is in Paris.',
 'Venus is the hottest planet in our solar system.']


## Step 5: Use the Retrieved Facts to Generate a Response


In [5]:

# Example query
query = "Tell me about the Eiffel Tower and space."

# Retrieve relevant facts
relevant_facts = retrieve_facts(query, facts)
relevant_facts

['The Eiffel Tower is in Paris.',
 'The Great Wall of China can be seen from space.']

In [6]:
# Combine relevant facts
context = " ".join(relevant_facts)
context

'The Eiffel Tower is in Paris. The Great Wall of China can be seen from space.'

In [7]:
prompt=f"{context} Based on these facts, {query}"
prompt

'The Eiffel Tower is in Paris. The Great Wall of China can be seen from space. Based on these facts, Tell me about the Eiffel Tower and space.'

In [8]:
# Generate a response using the OpenAI API
model ="gpt-3.5-turbo"
response = get_openai_response(client, prompt, model)
response

'The Eiffel Tower is a famous landmark located in Paris, France. Standing at 1,063 feet tall, it is one of the most recognizable structures in the world. However, despite its impressive height, the Eiffel Tower is not visible from space.\n\nOn the other hand, the Great Wall of China is a massive structure stretching over 13,000 miles across China. Due to its size and construction material, the Great Wall can be seen from space under certain conditions. Astronauts aboard the International Space Station have reported being able to see the Great Wall, particularly when passing over northern China.\n\nIn conclusion, while the Eiffel Tower is an iconic structure on Earth, it is not visible from space. On the other hand, the Great Wall of China is visible from space, showcasing the immense scale of this historical landmark.'

Document what you learned in Overleaf.

Will you use RAG any part of your project?